# Titanic: Machine Learning from Disaster

## Predict survival on the Titanic and get familiar with ML basics


The sinking of the RMS Titanic is one of the most infamous shipwrecks in history.  On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. This sensational tragedy shocked the international community and led to better safety regulations for ships.

One of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.

In this challenge, we ask you to complete the analysis of what sorts of people were likely to survive. In particular, we ask you to apply the tools of machine learning to predict which passengers survived the tragedy.


## Data Fields

  * **Survival** - Survival. 0 = No, 1 = Yes
  * **Pclass** - Ticket class. 1 = 1st, 2 = 2nd, 3 = 3rd
  * **Sex** - Sex.
  * **Age** - Age in years.
  * **SibSp** - # of siblings / spouses aboard the Titanic.
  * **Parch** - # of parents / children aboard the Titanic.
  * **Ticket** - Ticket number.
  * **Fare** - Passenger fare.
  * **Cabin** - Cabin number.
  * **Embarked** - Port of Embarkation. C = Cherbourg, Q = Queenstown, S = Southampton


In [1]:
import numpy as np
import pandas as pd

seed = 37

## Load Dataset

In [2]:
train = pd.read_csv("data/train.csv", index_col=["PassengerId"])

print(train.shape)
train.head()

(891, 11)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test = pd.read_csv("data/test.csv", index_col=["PassengerId"])

print(test.shape)
test.head()

(418, 10)


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## Preprocessing

### Merge train / test dataset

In [4]:
combi = pd.concat([train, test])

print(combi.shape)
combi.head()

(1309, 11)


,Age,Cabin,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Survived,Ticket
PassengerId,,,,,,,,,,,
1,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,3,male,1,0.0,A/5 21171
2,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,1,female,1,1.0,PC 17599
3,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,female,0,1.0,STON/O2. 3101282
4,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,1,female,1,1.0,113803
5,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,3,male,0,0.0,373450


### Encode Sex

In [5]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
combi["Sex_encode"] = encoder.fit_transform(combi["Sex"])

print(combi.shape)
combi[["Sex", "Sex_encode"]].head()

(1309, 12)


,Sex,Sex_encode
PassengerId,,
1,male,1
2,female,0
3,female,0
4,female,0
5,male,1


### Fill out missing fare

In [6]:
mean_fare = train["Fare"].mean()

print("Fare(Mean) = ${0:.3f}".format(mean_fare))

Fare(Mean) = $32.204


In [7]:
combi["Fare_fillout"] = combi["Fare"]

combi.loc[pd.isnull(combi["Fare"]), "Fare_fillout"] = mean_fare

missing_fare = combi[pd.isnull(combi["Fare"])]

print(missing_fare.shape)
combi.loc[missing_fare.index, ["Fare", "Fare_fillout"]].head()

(1, 13)


,Fare,Fare_fillout
PassengerId,,
1044,NaN,32.204208


### Encode Embarked

In [8]:
embarked = pd.get_dummies(combi["Embarked"], prefix="Embarked").astype(np.bool)

print(embarked.shape)
embarked.head()

(1309, 3)


,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,
1,False,False,True
2,True,False,False
3,False,False,True
4,False,False,True
5,False,False,True


In [21]:
combi["Family"] = combi["SibSp"] + combi["Parch"]

print(combi.shape)
combi.head(2)

(1309, 17)


,Age,Cabin,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Survived,Ticket,Sex_encode,Fare_fillout,Embarked_C,Embarked_Q,Embarked_S,Family
PassengerId,,,,,,,,,,,,,,,,,
1,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,3,male,1,0.0,A/5 21171,1,7.2500,False,False,True,1
2,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,1,female,1,1.0,PC 17599,0,71.2833,True,False,False,1


In [9]:
combi = pd.concat([combi, embarked], axis=1)

print(combi.shape)
combi[["Embarked", "Embarked_C", "Embarked_Q", "Embarked_S"]].head()

(1309, 16)


,Embarked,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,
1,S,False,False,True
2,C,True,False,False
3,S,False,False,True
4,S,False,False,True
5,S,False,False,True


### Split dataset into train / test

In [10]:
train = combi[pd.notnull(combi["Survived"])]

print(train.shape)
train.head()

(891, 16)


,Age,Cabin,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Survived,Ticket,Sex_encode,Fare_fillout,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,,,,,,
1,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,3,male,1,0.0,A/5 21171,1,7.2500,False,False,True
2,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,1,female,1,1.0,PC 17599,0,71.2833,True,False,False
3,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,female,0,1.0,STON/O2. 3101282,0,7.9250,False,False,True
4,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,1,female,1,1.0,113803,0,53.1000,False,False,True
5,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,3,male,0,0.0,373450,1,8.0500,False,False,True


In [11]:
test = combi[pd.isnull(combi["Survived"])]

test.drop("Survived", axis=1, inplace=True)

print(test.shape)
test.head()

(418, 15)


/Users/Rachel/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Age,Cabin,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Ticket,Sex_encode,Fare_fillout,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,,,,,
892,34.5,NaN,Q,7.8292,"Kelly, Mr. James",0,3,male,0,330911,1,7.8292,False,True,False
893,47.0,NaN,S,7.0000,"Wilkes, Mrs. James (Ellen Needs)",0,3,female,1,363272,0,7.0000,False,False,True
894,62.0,NaN,Q,9.6875,"Myles, Mr. Thomas Francis",0,2,male,0,240276,1,9.6875,False,True,False
895,27.0,NaN,S,8.6625,"Wirz, Mr. Albert",0,3,male,0,315154,1,8.6625,False,False,True
896,22.0,NaN,S,12.2875,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,3,female,1,3101298,0,12.2875,False,False,True


## Train

In [12]:
feature_names = ["Pclass", "Sex_encode", "Fare_fillout"]
feature_names = feature_names + list(embarked.columns)

label_name = "Survived"

In [13]:
X_train = train[feature_names]

print(X_train.shape)
X_train.head()

(891, 6)


,Pclass,Sex_encode,Fare_fillout,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,
1,3,1,7.2500,False,False,True
2,1,0,71.2833,True,False,False
3,3,0,7.9250,False,False,True
4,1,0,53.1000,False,False,True
5,3,1,8.0500,False,False,True


In [14]:
y_train = train[label_name]

print(y_train.shape)
y_train.head()

(891,)


PassengerId
1    0.0
2    1.0
3    1.0
4    1.0
5    0.0
Name: Survived, dtype: float64

In [15]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=5,
                               random_state=seed)

## Score

In [16]:
from sklearn.cross_validation import cross_val_score

%time score = cross_val_score(model, X_train, y_train, cv=100).mean()

print("Score = {0:.5f}".format(score))

/Users/Rachel/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


CPU times: user 370 ms, sys: 3.53 ms, total: 374 ms
Wall time: 373 ms
Score = 0.80675


## Predict

In [17]:
X_test = test[feature_names]

print(X_test.shape)
X_test.head()

(418, 6)


,Pclass,Sex_encode,Fare_fillout,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,
892,3,1,7.8292,False,True,False
893,3,0,7.0000,False,False,True
894,2,1,9.6875,False,True,False
895,3,1,8.6625,False,False,True
896,3,0,12.2875,False,False,True


In [18]:
model.fit(X_train, y_train)

prediction = model.predict(X_test)

print(prediction.shape)
prediction[:20]

(418,)


array([ 0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  1.,
        0.,  1.,  1.,  0.,  0.,  0.,  1.])

## Submit

In [19]:
submission = pd.read_csv("data/gender_submission.csv", index_col="PassengerId")

submission["Survived"] = prediction.astype(np.int32)

print(submission.shape)
submission.head()

(418, 1)


,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [20]:
submission.to_csv("data/04-decision-tree-pclass-fare-embarked-max-depth-5.csv")